# DBP downstream analysis

In [ ]:
source("/root/data/DBP_sa_bc/preprocess/utils.R")
setwd("/root/data/DBP_sa_bc/")
library(mclust) 
library(RColorBrewer)

# K <- 50 # break

parser <- ArgumentParser()
parser$add_argument("--task", type = "character", default = "wnn_rna")
parser$add_argument("--method", type = "character", default = "DBP_sa_bc")
parser$add_argument("--exp", type = "character", default = "e1")
parser$add_argument("--init_model", type = "character", default = "sp_00001899")
parser$add_argument("--K", type = "integer", default = "38")
o <- parser$parse_known_args()[[1]]

config <- parseTOML("configs/data.toml")[[o$task]]
subset_names <- basename(config$raw_data_dirs)
subset_ids <- sapply(seq_along(subset_names) - 1, toString)
input_dirs <- pj("result", o$task, o$exp, "default", "predict", o$init_model, paste0("subset_", subset_ids))
pp_dir <- pj("data", "processed", o$task)
output_dir <- pj("result", "analysis", o$task, o$method, o$exp)
break_index_dir <- pj("result", o$task, o$exp, "default", "predict", o$init_model)
mkdir(output_dir, remove_old = F)
label_paths <- pj(config$raw_data_dirs, "label", "meta.csv")

K <- o$K
dim_c <- parseTOML("configs/model.toml")[["default"]]$dim_c
qual_col_pals <- brewer.pal.info[brewer.pal.info$category == 'qual',]
dcols <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))
l <- 5  # figure size
L <- 8   # figure size
m <- 0.5  # legend margin


## Load model outputs

In [ ]:
z_list <- list()
w_list <- list()
rna_bc_list <- list()
cell_name_list <- list()
label_list1 <- list()
label_list2 <- list()
subset_name_list <- list()
S <- length(subset_names)
for (i in seq_along(subset_names)) {
    subset_name <- subset_names[i]
    z_dir       <- pj(input_dirs[i], "z", "joint")
    w_dir       <- pj(input_dirs[i], "w", "joint")
    rna_bc_dir  <- pj(input_dirs[i], "x_bc", "rna")
    fnames <- dir(path = z_dir, pattern = ".csv$")
    fnames <- str_sort(fnames, decreasing = F)

    z_subset_list <- list()
    w_subset_list <- list()
    rna_bc_subset_list <- list()

    N <- length(fnames)
    for (n in seq_along(fnames)) {
        message(paste0("Loading Subset ", i, "/", S, ", File ", n, "/", N))
        z_subset_list[[n]] <- read.csv(file.path(z_dir, fnames[n]), header = F)
        w_subset_list[[n]] <- read.csv(file.path(w_dir, fnames[n]), header = F)
        rna_bc_subset_list[[n]] <- read.csv(file.path(rna_bc_dir, fnames[n]), header = F)

    }
    z_list[[subset_name]] <- bind_rows(z_subset_list)
    w_list[[subset_name]] <- bind_rows(w_subset_list)
    rna_bc_list[[subset_name]] <- bind_rows(rna_bc_subset_list)

    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "cell_names.csv"), header = T)[, 2]
    if ("lung_ts" %in% o$task){
        label_list1[[subset_name]] <- read.csv(label_paths[i], header = T)[, "Celltypes1"]
        label_list2[[subset_name]] <- read.csv(label_paths[i], header = T)[, "Celltypes_updated_July_2020"]
    }else if("wnn_rna" %in% o$task){
        label_list1[[subset_name]] <- read.csv(label_paths[i], header = T)[, "celltype.l1"]
        label_list2[[subset_name]] <- read.csv(label_paths[i], header = T)[, "celltype.l2"]
    }else if("ga" %in% o$task){
        label_list1[[subset_name]] <- read.csv(label_paths[i], header = T)[, "celltype"]
        label_list2[[subset_name]] <- read.csv(label_paths[i], header = T)[,  "batch"]
    }else if("sim1" %in% o$task){
        label_list1[[subset_name]] <- read.csv(label_paths[i], header = T)[, "Group"]
        label_list2[[subset_name]] <- read.csv(label_paths[i], header = T)[, "Group"]
    }
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}

## Create seurat object

In [ ]:
rna_bc <- t(data.matrix(bind_rows(rna_bc_list)))
colnames(rna_bc) <- do.call("c", unname(cell_name_list))
rownames(rna_bc) <- read.csv(pj(pp_dir, "feat", "feat_names_rna.csv"), header = T)[, 2]
obj <- CreateSeuratObject(counts = rna_bc, assay = "rna_bc")

annotation <- GetGRangesFromEnsDb(EnsDb.Hsapiens.v86)
# seqlevelsStyle(annotation) <- "UCSC"
genome(annotation) <- "hg38"

z <- data.matrix(bind_rows(z_list))
w <- data.matrix(bind_rows(w_list))
c <- z[, 1:dim_c]*w

# break
index <- read.csv(pj(break_index_dir, "break_index.csv"), header = FALSE)
index <- index+1
names(index) <- "id"
tc <- data.frame(id = 1:dim(w)[2], y = t(c)) 
loc <- match(index$id,tc$id)
c_ord <- tc[loc,]
c_bre <- c_ord[1:K, !colnames(c_ord) %in% c("id") ]
emc <- data.matrix(t(c_bre))

# for umap
colnames(emc) <- paste0("F_", seq_len(ncol(emc)))
rownames(emc) <- colnames(obj)
obj[["emc"]] <- CreateDimReducObject(embeddings = emc, key = "F_", assay = "rna_bc")

# # for feature plot
# aemc <- data.matrix(t(abs(c_bre)))
# # aemc <- data.matrix(t(c_bre))
# colnames(aemc) <- paste0("F_", seq_len(ncol(aemc)))
# rownames(aemc) <- colnames(obj)
# obj[["aemc"]] <- CreateDimReducObject(embeddings = aemc, key = "F_", assay = "rna_bc")


obj@meta.data$celltype1 <- do.call("c", unname(label_list1))
obj@meta.data$celltype2 <- do.call("c", unname(label_list2))
obj@meta.data$batch <- do.call("c", unname(subset_name_list))
table(obj@meta.data$batch)[unique(obj@meta.data$batch)]

obj <- subset(obj, subset = nCount_rna_bc > 0)
obj

## Preprocess data

In [ ]:
# DefaultAssay(obj) <- "rna_bc"
# VariableFeatures(obj) <- rownames(obj)
# obj <-  NormalizeData(obj) %>%
#         # FindVariableFeatures(nfeatures = 2000) %>%
#         ScaleData()


## Visualize dimensionality reduction results

In [ ]:
obj <- RunUMAP(obj, reduction = 'emc', dims = 1:K, reduction.name = 'umap')
# SaveH5Seurat(obj, pj(output_dir, "obj_break.h5seurat"), overwrite = TRUE)

In [ ]:
if ("wnn_rna" %in% o$task){
    batch_cols <- col_8
    celltype1_cols <- col_8
    celltype2_cols <- col_31
}else if("lung_ts" %in% o$task){
    batch_cols <- col_5
    celltype1_cols <- col_16
    celltype2_cols <- col_28
}else if("ga" %in% o$task){
    batch_cols <- col_14
    celltype1_cols <- col_14
    celltype2_cols <- col_14
}else if("sim1" %in% o$task){
    batch_cols <- col_5
    celltype1_cols <- col_8
    celltype2_cols <- col_8
}

dim_plot(obj, w = L, h = L, reduction = "umap", no_axes = T,
    split.by = NULL, group.by = "batch", label = F, repel = T, 
    label.size = 4, pt.size = 0.1, cols = batch_cols, legend = F,
    save_path = pj(output_dir, paste(o$method, "merged_batch", sep = "_")))
     
dim_plot(obj, w = L, h = L, reduction = "umap", no_axes = T,
    split.by = NULL, group.by = "celltype1", label = F, repel = T, 
    label.size = 4, pt.size = 0.1, cols = celltype1_cols, legend = F,
    save_path = pj(output_dir, paste(o$method, "merged_label1", sep = "_")))

dim_plot(obj, w = L, h = L, reduction = "umap", no_axes = T,
    split.by = NULL, group.by = "celltype2", label = F, repel = T, 
    label.size = 4, pt.size = 0.1, cols = celltype2_cols, legend = F,
    save_path = pj(output_dir, paste(o$method, "merged_label2", sep = "_")))

dim_plot(obj, w = L*6, h = L, reduction = "umap", no_axes = T,
    split.by = "batch", group.by = "celltype1", label = F, repel = T, 
    label.size = 4, pt.size = 0.1, cols = celltype1_cols, legend = F,
    save_path = pj(output_dir, paste(o$method, "batch_split1", sep = "_"))) 

dim_plot(obj, w = L*6, h = L, reduction = "umap", no_axes = T,
    split.by = "batch", group.by = "celltype2", label = F, repel = T, 
    label.size = 4, pt.size = 0.1, cols = celltype2_cols, legend = F,
    save_path = pj(output_dir, paste(o$method, "batch_split2", sep = "_"))) 